In [1]:
from dexperts import DExperts

/Users/eliott/Desktop/bias-free-nlg/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dexperts = DExperts(
    base_model='gpt2-medium',
    antiexpert_model='eliolio/gpt2-finetuned-redditbias',
    expert_model='eliolio/gpt2-finetuned-reddit-antibias',
    tokenizer='gpt2',
)

dexperts_void = DExperts(
    base_model='gpt2-medium',
    tokenizer='gpt2',
)

In [3]:
def generate_text(prompt, typical_p=None, top_p=None):
    inputs = dexperts.tokenizer(prompt, return_tensors='pt')
    out = dexperts.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        num_return_sequences=3,
        max_new_tokens=15,
        do_sample=True,
        top_p=top_p,
        typical_p=typical_p,
    )
    return [dexperts.tokenizer.decode(out[i], skip_special_tokens=True) for i in range(len(out))]

In [4]:
generate_text(prompt='I am very')

['I am very quiet," Shatner said at the conference. "I don\'t quiet',
 'I am very sorry," said Obama. "But, I happen to respect the quiet quiet',
 "I am very confident in our ability to defend the university to protect students,' said university Chancellor"]

In [7]:
res = dexperts("I am very", alpha=0.0)
res

{'logits': tensor([[[ -78.3232,  -77.6039,  -80.8931,  ...,  -89.6521,  -82.6672,
            -80.4394],
          [-100.7663, -100.0332, -105.9358,  ..., -109.0896, -108.7145,
           -102.2539],
          [-118.6408, -118.0217, -124.4341,  ..., -125.5256, -125.2729,
           -120.2754]]]),
 'perplexity': tensor(1828.4690),
 'encoded_text': tensor([[ 40, 716, 845]])}

In [12]:
res['encoded_text']

tensor([[ 40, 716, 845]])

In [17]:
import torch
torch.softmax(res['logits'], dim=-1)

tensor([[[1.2388e-04, 2.5432e-04, 9.4824e-06,  ..., 1.4891e-09,
          1.6086e-06, 1.4926e-05],
         [2.4473e-04, 5.0938e-04, 1.3919e-06,  ..., 5.9420e-08,
          8.6463e-08, 5.5289e-05],
         [1.4122e-05, 2.6227e-05, 4.3041e-08,  ..., 1.4450e-08,
          1.8602e-08, 2.7542e-06]]])

In [14]:
torch.softmax(res['encoded_text'][0], dim=-1).argmax()

RuntimeError: "softmax_lastdim_kernel_impl" not implemented for 'Long'

In [19]:
res['encoded_text'][0].item()

ValueError: only one element tensors can be converted to Python scalars

In [6]:
res['logits']

tensor([[[ -78.3232,  -77.6039,  -80.8931,  ...,  -89.6521,  -82.6672,
           -80.4394],
         [-100.7663, -100.0332, -105.9358,  ..., -109.0896, -108.7145,
          -102.2539],
         [-118.6408, -118.0217, -124.4341,  ..., -125.5256, -125.2729,
          -120.2754]]])

In [23]:
dexperts_void("I am very", alpha=10.0)

{'logits': tensor([[[ -78.3232,  -77.6039,  -80.8931,  ...,  -89.6521,  -82.6672,
            -80.4394],
          [-100.7663, -100.0332, -105.9358,  ..., -109.0896, -108.7145,
           -102.2539],
          [-118.6408, -118.0217, -124.4341,  ..., -125.5256, -125.2729,
           -120.2754]]]),
 'perplexity': tensor(1828.4690)}